<a href="https://colab.research.google.com/github/blkeyd/CVPR/blob/main/Final/Paper/AlexNet%2BGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AlexNet + GAN Augmented Training

Importing Libraries

In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, ConcatDataset
from PIL import Image
from torch.utils.data import Dataset

Mount Drive

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Device

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


Path

In [11]:
real_fire_dir = "/content/drive/MyDrive/fire_dataset_split/train/fire"
gan_fire_dir  = "/content/drive/MyDrive/gan_generated_fire"
nofire_dir    = "/content/drive/MyDrive/fire_dataset_split/train/nofire"
val_dir       = "/content/drive/MyDrive/fire_dataset_split/val"
test_dir      = "/content/drive/MyDrive/fire_dataset_split/test"

Transforms

In [12]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

Custom Dataset for GAN images

In [13]:
class GANFireDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path,f)
                       for f in os.listdir(folder_path) if f.endswith(".png")]
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        img = Image.open(self.images[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = 0  # fire class
        return img, label

Load datasets

In [14]:
# Real fire images
real_fire_dataset = datasets.ImageFolder(os.path.dirname(real_fire_dir), transform=transform)
fire_class_index = real_fire_dataset.class_to_idx['fire']
real_fire_dataset.samples = [s for s in real_fire_dataset.samples if s[1]==fire_class_index]

# GAN fire images
gan_fire_dataset = GANFireDataset(gan_fire_dir, transform=transform)

# No-fire images
nofire_dataset = datasets.ImageFolder(os.path.dirname(nofire_dir), transform=transform)
nofire_class_index = nofire_dataset.class_to_idx['nofire']
nofire_dataset.samples = [s for s in nofire_dataset.samples if s[1]==nofire_class_index]

# Merge datasets
train_dataset = ConcatDataset([real_fire_dataset, gan_fire_dataset, nofire_dataset])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Validation & Test datasets
val_dataset = datasets.ImageFolder(val_dir, transform=transform)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

Load Standart AlexNet

In [15]:
model = models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(4096, 2)
model = model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


100%|██████████| 233M/233M [00:01<00:00, 184MB/s]


Loss and Optimizer

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Training Function

In [17]:
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct = 0
        total = 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
        train_acc = 100*correct/total

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, preds = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (preds == labels).sum().item()
        val_acc = 100*val_correct/val_total

        print(f"Epoch [{epoch+1}/{epochs}] Train Acc: {train_acc:.2f}%  Val Acc: {val_acc:.2f}%")

Evaluation

In [18]:
from sklearn.metrics import confusion_matrix, classification_report
def evaluate_model(model, test_loader):
    model.eval()
    all_labels, all_preds = [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs,1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    print("Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))
    print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=['fire','nofire']))

Train and Evaluate

In [19]:
train_model(model, train_loader, val_loader, epochs=10)
evaluate_model(model, test_loader)

Epoch [1/10] Train Acc: 95.74%  Val Acc: 98.66%
Epoch [2/10] Train Acc: 99.67%  Val Acc: 98.66%
Epoch [3/10] Train Acc: 99.92%  Val Acc: 99.33%
Epoch [4/10] Train Acc: 100.00%  Val Acc: 99.33%
Epoch [5/10] Train Acc: 100.00%  Val Acc: 99.33%
Epoch [6/10] Train Acc: 100.00%  Val Acc: 99.33%
Epoch [7/10] Train Acc: 100.00%  Val Acc: 98.66%
Epoch [8/10] Train Acc: 100.00%  Val Acc: 97.99%
Epoch [9/10] Train Acc: 100.00%  Val Acc: 97.99%
Epoch [10/10] Train Acc: 100.00%  Val Acc: 97.99%
Confusion Matrix:
 [[110   4]
 [  1  37]]

Classification Report:
               precision    recall  f1-score   support

        fire       0.99      0.96      0.98       114
      nofire       0.90      0.97      0.94        38

    accuracy                           0.97       152
   macro avg       0.95      0.97      0.96       152
weighted avg       0.97      0.97      0.97       152



Save Model

In [20]:
torch.save(model.state_dict(), "/content/drive/MyDrive/alexnet_gan_fire.pth")
print("Traditional AlexNet + GAN model saved.")

Traditional AlexNet + GAN model saved.
